In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
#settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: c:\Users\lulev\Desktop\KTH\mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
city_dict = json.loads(location_str)

# city=location['city']
# aqicn_url=location['aqicn_url']
# latitude=location['latitude']
# longitude=location['longitude']

today = datetime.date.today()

city_dict

2025-11-15 12:44:56,685 INFO: Initializing external client
2025-11-15 12:44:56,685 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-15 12:44:58,364 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279155


{'bodo': {'url': 'https://api.waqi.info/feed/@12706',
  'csv_file': 'c:\\Users\\lulev\\Desktop\\KTH\\mlfs-book/data/bodo.csv',
  'latitude': 67.28,
  'longitude': 14.38},
 'moirana': {'url': 'https://api.waqi.info/feed/@12698',
  'csv_file': 'c:\\Users\\lulev\\Desktop\\KTH\\mlfs-book/data/moirana.csv',
  'latitude': 66.33,
  'longitude': 14.18}}

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

df_list = []
for city in city_dict:
    temp = util.get_pm25(city_dict[city]["url"], city, today, AQICN_API_KEY)
    # aq_today_df["lag1"] = # featuregroup("pm25", today-1day)
    # aq_today_df["lag2"] = # featuregroup("pm25", today-2day)
    df_list.append(temp)
aq_today_df = pd.concat(df_list, ignore_index=True)
aq_today_df

,pm25,city,date,url
0,10.0,bodo,2025-11-15,https://api.waqi.info/feed/@12706
1,5.0,moirana,2025-11-15,https://api.waqi.info/feed/@12698


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   pm25    2 non-null      float32       
 1   city    2 non-null      object        
 2   date    2 non-null      datetime64[ns]
 3   url     2 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
weather_list = []
for city in city_dict:
    weather_temp = util.get_hourly_weather_forecast(city, city_dict[city]["latitude"], city_dict[city]["longitude"])
    weather_temp["city"] = city
    weather_list.append(weather_temp)
hourly_df = pd.concat(weather_list, ignore_index=True)

hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df

Coordinates 67.25°N 14.5°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 66.25°N 14.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-15,3.85,1.5,25.842384,251.312546,bodo
1,2025-11-16,-1.40,2.0,8.350138,172.568680,bodo
2,2025-11-17,-1.80,0.0,1.800000,143.130020,bodo
3,2025-11-18,-3.30,0.3,6.120000,151.927612,bodo
4,2025-11-19,-6.90,0.0,11.874544,104.036270,bodo
5,2025-11-20,-2.15,0.3,7.342588,168.690109,bodo
6,2025-11-21,-3.80,0.1,10.464797,116.564987,bodo
7,2025-11-15,2.50,1.9,15.546833,222.184372,moirana
8,2025-11-16,-3.30,0.8,0.804985,243.435013,moirana
9,2025-11-17,-8.60,0.0,0.509117,224.999893,moirana


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         14 non-null     datetime64[ns]
 1   temperature_2m_mean          14 non-null     float32       
 2   precipitation_sum            14 non-null     float32       
 3   wind_speed_10m_max           14 non-null     float32       
 4   wind_direction_10m_dominant  14 non-null     float32       
 5   city                         14 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 576.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-15 12:45:04,030 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718620


Uploading Dataframe: 100.00% |██████████| Rows 2/2 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758797
         }
       },
       "result": {
         "observed_value": 5.0,
         "element_count": 2,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T11:45:04.000030Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-15 12:45:17,829 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718621


Uploading Dataframe: 100.00% |██████████| Rows 14/14 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-15 12:45:35,160 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 12:45:38,345 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 12:47:20,579 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 12:47:20,751 INFO: Waiting for log aggregation to finish.
2025-11-15 12:47:32,941 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758798
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 14,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T11:45:17.000828Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
